# Violence against women

In [1]:
import os; os.chdir('..')
%load_ext autoreload
%autoreload 2

In [10]:
import unidecode, numpy as np, pandas as pd
from pathlib import Path

INTERIM = Path('data/interim/criminality/vaw-endes/')
PROC = Path('data/processed/criminality/')

# Read File

In [3]:
!ls {INTERIM}

consecuencias_violencia_fisica_compañero.csv
victimas_buscaron_ayuda_institucion.csv
victimas_buscaron_ayuda_personas.csv
violencia_fisica_compañero.csv
violencia_fisica_otro.csv
violencia_psicologica_fisica_sexual_compañero.csv
violencia_psicologica_verbal_compañero.csv
violencia_sexual_compañero.csv


In [4]:
def read_file(file):
    df = pd.read_csv(file, encoding='latin1', low_memory=False).iloc[:-6]
    cat = df.columns[0]
    if df.iloc[3].notna().sum() > 1:
        df.columns = ['location', 'variable'] + df.iloc[2, 2:].tolist()
        df['variable'] = f'{cat} / ' + df['variable']
    else:
        df.columns = ['location'] + df.iloc[2, 1:].tolist()
        df['variable'] = cat
    
    df = (df.drop('Periodo', axis=1).iloc[4:].
             reset_index(drop=True).
             fillna(method='pad').
             melt(['location', 'variable'], var_name='year'))
    df.value = df.value.str.replace(',', '.').apply(lambda x: x if pd.isna(x) else float(x[:-1])/100)
    
    df.location = (df.location.
                      str.replace('PROVINCIA DE LIMA 2/', 'LIMA').
                      str.replace('LIMA 1/', 'LIMA/CALLAO').
                      str.replace('REGIÓN LIMA 3/', 'LIMA PROVINCIAS').
                      str.replace('PROV. CONST. DEL CALLAO', 'CALLAO').
                      str.title().
                      str.replace(' De ', ' de '))
    
    return df

In [5]:
df = pd.concat([read_file(e) for e in INTERIM.iterdir()], ignore_index=True)
df.head()

,location,variable,year,value
0,Nacional,MUJERES DE 15 A 49 AÑOS DE EDAD QUE BUSCARON A...,2009,0.236
1,Amazonas,MUJERES DE 15 A 49 AÑOS DE EDAD QUE BUSCARON A...,2009,0.199
2,Áncash,MUJERES DE 15 A 49 AÑOS DE EDAD QUE BUSCARON A...,2009,0.252
3,Apurímac,MUJERES DE 15 A 49 AÑOS DE EDAD QUE BUSCARON A...,2009,0.336
4,Arequipa,MUJERES DE 15 A 49 AÑOS DE EDAD QUE BUSCARON A...,2009,0.337


In [12]:
PROC.mkdir(exist_ok=True)
df.to_csv(PROC/'vaw-endes.csv')

In [13]:
!ls -lh {PROC}

total 488K
-rw-rw-r-- 1 kopita kopita 8.5K May  6 15:00 population_inei.csv
-rw-rw-r-- 1 kopita kopita  82K May  6 15:00 tipo-ubicacion.csv
-rw-rw-r-- 1 kopita kopita 389K Jun  4 00:33 vaw-endes.csv
